In [1]:
import pandas as pd
import sklearn

from sklearn.feature_selection import SequentialFeatureSelector


In [2]:
# Stepwise Selection

def forward_stepwise_selection(X, y, f_names):
    sfs_forward = SequentialFeatureSelector(linear_model.LinearRegression(), 
                                        n_features_to_select=5, 
                                        direction = 'forward').fit(X,y)
    print('support: ', sfs_forward.get_support(), "\n")
    selected = sfs_forward.get_support(indices = True)
    print(selected)
    print('Selected input features using Forward Stepwise Selection: \n', f_names[selected])
    
    return f_names[selected]

def backward_stepwise_selection(X, y, f_names):
    sfs_backward = SequentialFeatureSelector(linear_model.LinearRegression(), 
                                        n_features_to_select=5, 
                                        direction = 'backward').fit(X,y)
    print('support: ', sfs_backward.get_support(), "\n")
    selected = sfs_backward.get_support(indices = True)
    print(selected)
    print('Selected input features using Forward Stepwise Selection: \n', f_names[selected])
    
    return f_names[selected]

In [3]:
# Regressors

# Random Forest Regressor
def random_forest_reg(X_train, y_train, X_test, y_test):
    clf_RF = RandomForestRegressor(n_estimators = 100, max_depth = 4, max_features = 2, random_state = 7)
    clf_RF = clf_RF.fit(X_train, y_train)
    y_predict = clf_RF.predict(X_test)
    mse_RF = mean_squared_error(y_test, y_predict)
    r2_RF = r2_score(y_test, y_predict)
    print(mse_RF)
    print(r2_RF)
    return r2_RF, y_predict

# Gradient Boosting Regressor
def gradient_boosting_reg(X_train, y_train, X_test, y_test):
    clf_GB = GradientBoostingRegressor(n_estimators=1000, learning_rate=0.1, max_depth=1, random_state=7)
    clf_GB = clf_GB.fit(X_train, y_train)
    y_predict = clf_GB.predict(X_test)
    mse_GB = mean_squared_error(y_test, y_predict)
    r2_GB = r2_score(y_test, y_predict)
    print(mse_GB)
    print(r2_GB)
    
    return r2_GB
    
# Bagging Regressor
def bagging_reg():
    estimator = DecisionTreeRegressor(max_depth=4)
    clf_bag = BaggingRegressor(base_estimator=estimator, n_estimators=100, random_state=7)
    clf_bag = clf_bag.fit(X_train, y_train)
    y_predict = clf_bag.predict(X_test)
    mse_bag = mean_squared_error(y_test, y_predict)
    r2_bag = r2_score(y_test, y_predict)
    print(mse_bag)
    print(r2_bag)
    
    return r2_bag

In [ ]:
def difference(list1, list2):
    return list(set(list1).symmetric_difference(set(list2)))

In [ ]:
import unittest
class TestCost(unittest.TestCase):
    """
    test cases
    """
    # stepwise selection
    def test_one_shot_test_forward_stepwise(self):
        '''
        Forward stepwise selection
        '''
        # add X and y
        names = 'systemSizeInDCSTC_KW_', 'Up_FrontCashIncentive___', 'azimuth_1', 'tilt_1', 'mod_BIPV1', 'mod_bifacial1', 'mod_nameplate_capacity1', 'mod_efficiency1', 'inverterQuantity_1', 'inv_microinv1', 'inv_battery_hybrid1', 'inv_builtin_meter1', 'inv_outputcapacity1', 'dc_optimizer', 'ILR', 'TotalModuleQty', 'latitude', 'longitude'
        f_names = np.array(names)
        stepwise_selection(X, y, f_names)
        return
    
    def test_forward_stepwise_input(self):
        # add dataset
        # check if string is present
        self.assertTrue(all(isinstance(x, (int, float)) for x in df))
        return
        
    def test_one_shot_test_forward_stepwise(self):
        '''
        Backward stepwise selection
        '''
        # add X and y
        names = 'systemSizeInDCSTC_KW_', 'Up_FrontCashIncentive___', 'azimuth_1', 'tilt_1', 'mod_BIPV1', 'mod_bifacial1', 'mod_nameplate_capacity1', 'mod_efficiency1', 'inverterQuantity_1', 'inv_microinv1', 'inv_battery_hybrid1', 'inv_builtin_meter1', 'inv_outputcapacity1', 'dc_optimizer', 'ILR', 'TotalModuleQty', 'latitude', 'longitude'
        f_names = np.array(names)
        stepwise_selection(X, y, f_names)
        return
    
    def test_backward_stepwise_input(self):
        # add dataset
        # check if string is present
        self.assertTrue(all(isinstance(x, (int, float)) for x in df))
        return
    
    # random forest regressor
    def test_one_shot_rf(self):
        self.assertTrue(random_forest_reg(X_train, y_train, X_test, y_test))
        return
    


suite = unittest.TestLoader().loadTestsFromTestCase(TestPairwiseCorrelation)
_ = unittest.TextTestRunner().run(suite)